# BERT text classification: data validation of collected job description details.

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from datasets import DatasetDict, Dataset
from transformers import TFAutoModel, AutoTokenizer
import tensorflow as tf
from keras import backend as K
import tensorflow_addons as tfa
import numpy as np

c:\Users\temulenbd\anaconda3\Lib\site-packages\tensorflow_addons\utils\tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(
c:\Users\temulenbd\anaconda3\Lib\site-packages\tensorflow_addons\utils\ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.12.0 and strictly below 2.15.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.15.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make

In [2]:
# Import finalized dataset as pandas data frame.
df = pd.read_csv('data_jobads_final.csv', index_col=None)

# Replace newline characters in the 'job_description' column with a space.
df['job_description'] = df['job_description'].str.replace('\n', ' ')

# Remove None values.
df = df.dropna()

# Select and use only the last two columns for this evaluation.
df = df.iloc[:,-2:]

df.head(3)

,job_description,label
0,silver stream healthcare group offer great emp...,registered_nurse
1,create a better future for yourself !! recruit...,registered_nurse
2,"access healthcare, one of ireland’s leading he...",registered_nurse


In [3]:
# Updated 'label' column values according to the specified mapping.
mapping = {'registered_nurse': 0, 'electrician': 1, 'data_analyst': 2}
df['label_text'] = df['label']
df['label'] = df['label'].replace(mapping) # or df['label'] = pd.factorize(df.label)[0]

df['label'].value_counts()

0    644
2    376
1    146
Name: label, dtype: int64

In [4]:
# Split the DataFrame into training and testing sets while maintaining label proportions.
train, validation = train_test_split(df, test_size=0.3, random_state=820, stratify=df['label'])

print('The shape of the TRAINING dataset is:', train.shape)
print('The shape of the VALIDATION dataset is:', validation.shape)

The shape of the TRAINING dataset is: (816, 3)
The shape of the VALIDATION dataset is: (350, 3)


In [5]:
# Convert DataFrames to Hugging Face Dataset.
train_dataset = Dataset.from_pandas(train)
val_dataset = Dataset.from_pandas(validation)

# Remove '__index_level_0__' feature
train_dataset = train_dataset.remove_columns('__index_level_0__')
val_dataset = val_dataset.remove_columns('__index_level_0__')

# Create DatasetDict.
jobads = DatasetDict({
    'train': train_dataset,
    'validation': val_dataset,
    })

In [6]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
model = TFAutoModel.from_pretrained('bert-base-uncased')

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [7]:
def tokenize(batch):
    return tokenizer(batch['job_description'], padding=True, truncation=True)

jobads_encoded = jobads.map(tokenize, batched=True, batch_size=None)

Map:   0%|          | 0/816 [00:00<?, ? examples/s]

Map:   0%|          | 0/350 [00:00<?, ? examples/s]

In [8]:
jobads_encoded

DatasetDict({
    train: Dataset({
        features: ['job_description', 'label', 'label_text', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 816
    })
    validation: Dataset({
        features: ['job_description', 'label', 'label_text', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 350
    })
})

In [9]:
jobads_encoded.set_format('tf',
                          columns=['input_ids', 'attention_mask', 'token_type_ids', 'label'])

BATCH_SIZE = 32

def order(inp):
    '''
    This function will group all the inputs of BERT into single
    dictionary and then output it with labels.
    '''
    data = list(inp.values())
    return {
        'input_ids' : data[1],
        'attention_mask' : data[2],
        'token_type_ids' : data[3]
    }, data[0]
    
# Convert train split of 'jobads_encoded' to tensorflow format.
train_dataset = tf.data.Dataset.from_tensor_slices(jobads_encoded['train'][:])

# Set batch_size and shuffle.
train_dataset = train_dataset.batch(BATCH_SIZE).shuffle(100)

# Map the 'order' function.
train_dataset = train_dataset.map(order, num_parallel_calls=tf.data.AUTOTUNE)

# ... doing the same for test set ...
val_dataset = tf.data.Dataset.from_tensor_slices(jobads_encoded['validation'][:])
val_dataset = val_dataset.batch(BATCH_SIZE)
val_dataset = val_dataset.map(order, num_parallel_calls=tf.data.AUTOTUNE)

In [10]:
inp, out = next(iter(train_dataset)) # a batch from train_dataset
print(inp, '\n\n', out)

{'input_ids': <tf.Tensor: shape=(32, 512), dtype=int64, numpy=
array([[  101,  3105,  6412, ...,     0,     0,     0],
       [  101,  2057,  2024, ...,     0,     0,     0],
       [  101, 15563,  2024, ...,     0,     0,     0],
       ...,
       [  101,  2307,  3105, ...,     0,     0,     0],
       [  101, 15563,  2024, ...,     0,     0,     0],
       [  101,  1996,  4456, ...,     0,     0,     0]], dtype=int64)>, 'attention_mask': <tf.Tensor: shape=(32, 512), dtype=int64, numpy=
array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)>, 'token_type_ids': <tf.Tensor: shape=(32, 512), dtype=int64, numpy=
array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]], dtype=int64)>}

In [11]:
class BERTForClassification(tf.keras.Model):
    
    def __init__(self, bert_model, num_classes):
        super().__init__()
        self.bert = bert_model
        self.fc = tf.keras.layers.Dense(num_classes, activation='softmax')
        
    def call(self, inputs):
        x = self.bert(inputs)[1]
        return self.fc(x)

In [12]:
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + 
    K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [13]:
classifier = BERTForClassification(model, num_classes=3)

classifier.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy', recall_m, precision_m, f1_m]
)

In [14]:
history = classifier.fit(
    train_dataset,
    epochs=3
)

Epoch 1/3


26/26 [==============================] - 1780s 65s/step - loss: 0.8921 - accuracy: 0.6225 - recall_m: 1.2793 - precision_m: 0.6237 - f1_m: 0.8270
Epoch 2/3
26/26 [==============================] - 1654s 59s/step - loss: 0.5341 - accuracy: 0.8162 - recall_m: 1.0906 - precision_m: 0.4977 - f1_m: 0.6763
Epoch 3/3
26/26 [==============================] - 1463s 56s/step - loss: 0.1203 - accuracy: 0.9718 - recall_m: 1.0537 - precision_m: 0.4818 - f1_m: 0.6544


In [15]:
#evaluation_results = classifier.evaluate(val_dataset)

In [16]:
'''print('All Metrics:')
for metric_name, metric_value in zip(classifier.metrics_names, evaluation_results):
    print(f"{metric_name}: {metric_value:.4f}")'''


'print(\'All Metrics:\')\nfor metric_name, metric_value in zip(classifier.metrics_names, evaluation_results):\n    print(f"{metric_name}: {metric_value:.4f}")'

In [17]:
classifier.save('fine_tuned_bert_multi_label_classifier_temuulen')

INFO:tensorflow:Assets written to: fine_tuned_bert_multi_label_classifier_temuulen\assets


INFO:tensorflow:Assets written to: fine_tuned_bert_multi_label_classifier_temuulen\assets


In [18]:
'''classifier.fit(train_dataset, epochs=3, validation_data=(val_data, val_labels))

# After training, you can evaluate on the test set and get the confusion matrix
test_loss, test_accuracy, test_precision, test_recall, test_f1 = classifier.evaluate(test_data, test_labels)
print(f'Test Accuracy: {test_accuracy}')
print(f'Test Precision: {test_precision}')
print(f'Test Recall: {test_recall}')
print(f'Test F1 Score: {test_f1}')

# Get confusion matrix
predictions = classifier.predict(test_data)
conf_matrix = confusion_matrix(test_labels, np.argmax(predictions, axis=-1))
print('Confusion Matrix:')
print(conf_matrix)'''

"classifier.fit(train_dataset, epochs=3, validation_data=(val_data, val_labels))\n\n# After training, you can evaluate on the test set and get the confusion matrix\ntest_loss, test_accuracy, test_precision, test_recall, test_f1 = classifier.evaluate(test_data, test_labels)\nprint(f'Test Accuracy: {test_accuracy}')\nprint(f'Test Precision: {test_precision}')\nprint(f'Test Recall: {test_recall}')\nprint(f'Test F1 Score: {test_f1}')\n\n# Get confusion matrix\npredictions = classifier.predict(test_data)\nconf_matrix = confusion_matrix(test_labels, np.argmax(predictions, axis=-1))\nprint('Confusion Matrix:')\nprint(conf_matrix)"